# Bonobo ETL


In [ ]:
# injectable objects
import bonobo
def get_services(**kwargs):
    return {
        'datareader': bonobo.CsvReader(
            delimiter='|',
            fields = [ 'id', 'city', 'date', 'actual_mean_temp', 'actual_min_temp', 'actual_max_temp', 'actual_precipitation', 'average_precipitation', 'record_precipitation', 'reserved2' ]
            ),
        'filename': kwargs['filename']
    }

Once services have been defined, add the functions involved in the process.

In [ ]:
from bonobo.config import use

def arg0_to_kwargs(row):
    return bonobo.Bag(**row)

@use('filename')
@use('datareader')
def read_recs(filename, datareader):
    print(type(datareader))
    for rec in datareader.read(filename):
        yield rec

def transform(datain):
    print(f'transform ...')
    return datain

def write_recs(data):
    print(f'write_recs', data)
    pass

Finally, define the graph and run it.

In [ ]:
graph = bonobo.Graph(
    read_recs,
    transform,
    arg0_to_kwargs,
    bonobo.PrettyPrinter())
#    write_recs)
bonobo.run(graph, services=get_services(filename='KPHX.dat'))

In [ ]:
reader = bonobo.CsvReader(
    delimiter='|',
    fields=['id', 'city', 'date', 'actual_mean_temp', 'actual_min_temp', 'actual_max_temp',
            'actual_precipitation', 'average_precipitation', 'record_precipitation', 'reserved2']
)
print(type(reader))
for l in reader.read(file='KPHX.dat'):
    print(l)
